## fastai Abhishek Inference Module

This will be an inference module based on my previous notebook [here](https://www.kaggle.com/muellerzr/recreating-abhishek-s-tez-with-fastai)

We'll be showing how to import our exported learner, perform a 15x TTA (as is done with his notebook), as well as recreate our transform pipeline

## Our Imports

Let's grab all our imports

In [1]:
%cd ../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master
from efficientnet_pytorch import EfficientNet
%cd -

/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master
/kaggle/working


In [2]:
from fastai.vision.all import *

## Bringing in the Items We Need

When we export a `Learner` object, it expects *everything* we had be available when we load it back in. Specifically our **functions** should be in the available namespace. Below we've done just that, bringing back `AlbumentationsTransform`, our `get_x` and `get_y`, and our `LeafModel`:

In [3]:
class AlbumentationsTransform(RandTransform):
    "A transform handler for multiple `Albumentation` transforms"
    split_idx,order=None,2
    def __init__(self, train_aug, valid_aug): store_attr()
    
    def before_call(self, b, split_idx):
        self.idx = split_idx
    
    def encodes(self, img: PILImage):
        if self.idx == 0:
            aug_img = self.train_aug(image=np.array(img))['image']
        else:
            aug_img = self.valid_aug(image=np.array(img))['image']
        return PILImage.create(aug_img)

In [4]:
def get_x(row): return data_path/row['image_id']
def get_y(row): return row['label']

In [5]:
class LeafModel(Module):
    def __init__(self, num_classes):

        self.effnet = EfficientNet.from_pretrained("efficientnet-b3")
        self.dropout = nn.Dropout(0.1)
        self.out = nn.Linear(1536, num_classes)

    def forward(self, image):
        batch_size, _, _, _ = image.shape

        x = self.effnet.extract_features(image)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        outputs = self.out(self.dropout(x))
        return outputs

We'll also need that `data_path`

In [6]:
path = Path("../input")
data_path = path/'cassava-leaf-disease-classification'

And now we can load our model back in with a simple `load_learner`

In [7]:
learn = load_learner(Path('../input/abishektez/baseline'), cpu=False)

## Performing TTA

For our TTA we will be doing a 15x ensemble similar to what was performed there. 

> Inference code is based off my kernel [here](https://www.kaggle.com/muellerzr/submission-notebook)

In [8]:
learn.to_native_fp32()

In [9]:
sample_df = pd.read_csv(data_path/'sample_submission.csv')
sample_df.head()

,image_id,label
0,2216849948.jpg,4


In [10]:
sample_copy = sample_df.copy()
sample_copy['image_id'] = sample_copy['image_id'].apply(lambda x: f'test_images/{x}')

Let's build our `test_dl` and grab our predictions:

In [11]:
test_dl = learn.dls.test_dl(sample_copy)

In [12]:
preds, _ = learn.tta(dl=test_dl, n=15, beta=0)

In [13]:
sample_df['label'] = preds.argmax(dim=-1).numpy()

In [14]:
sample_df.to_csv('submission.csv',index=False)